In [1]:
import os
os.chdir(os.path.dirname(os.path.abspath('__file__')))
os.chdir("./")
print(os.getcwd())

/home/siu/gmxtools/jupyter


In [2]:
# pip install wget
# conda install -c conda-forge biopython nglview
# Reload Browser

In [3]:
# 1단계: PDB 파일 다운로드
import wget
from Bio.PDB import PDBParser
import nglview as nv

# 예시 PDB ID
pdb_id = "6cu7"  # 원하는 PDB ID를 여기에 입력

# wget 명령어 실행
url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
pdb_file = f"{pdb_id}.pdb"
filename = wget.download(url, pdb_file)

print(f"Downloaded {filename}")

-1 / unknownDownloaded 6cu7.pdb


In [4]:
# PDB 파일 로드 및 구조 시각화
parser = PDBParser(QUIET=True)
structure = parser.get_structure('structure', f"{pdb_id}.pdb")

# NGLView로 단백질 구조 시각화
view = nv.show_biopython(structure)
view.add_representation('cartoon')
view

NGLWidget()

In [5]:
from Bio.PDB import PDBParser
import numpy as np

# 2단계: 두 체인의 센트로이드 간 거리 측정

# PDB 파일 로드
parser = PDBParser(QUIET=True)
structure = parser.get_structure('structure', f"{pdb_id}.pdb")

# 체인 A와 B 선택
chainA = structure[0]['B']  # 원하는 체인 ID로 변경 가능
chainB = structure[0]['D']  # 원하는 체인 ID로 변경 가능

# 센트로이드 계산 함수
def calculate_centroid(chain):
    coords = [atom.coord for atom in chain.get_atoms()]
    centroid = np.mean(coords, axis=0)
    return centroid

# 체인 A와 B의 센트로이드 계산
centroidA = calculate_centroid(chainA)
centroidB = calculate_centroid(chainB)

# 두 센트로이드 간의 거리 계산
interchain_distance = np.linalg.norm(centroidA - centroidB)
print(f"Interchain Distance: {interchain_distance} Å")


Interchain Distance: 4.855152130126953 Å


In [6]:
from Bio.PDB import PDBIO

# 3단계: 축 방향으로 구조 이동

# n 값과 축 방향 지정
n_dist = 5  # 원하는 n 값을 여기에 입력
axis = 'z'  # 이동시킬 축 ('x', 'y', 'z')

# 축 인덱스 설정
axis_index = {'x': 0, 'y': 1, 'z': 2}[axis]

# 구조 이동
translation_distance = interchain_distance * n_dist
for atom in structure.get_atoms():
    atom.coord[axis_index] += translation_distance

# 이동된 구조를 새로운 PDB 파일로 저장
output_translated_pdb = f"{pdb_id}_translated.pdb"
io = PDBIO()
io.set_structure(structure)
io.save(output_translated_pdb)

print(f"Translated PDB saved to: {output_translated_pdb}")


Translated PDB saved to: 6cu7_translated.pdb


In [7]:
from Bio.PDB import PDBParser, PDBIO
import string

# 4단계: 체인 이름 변경

# 원본 구조 로드
original_structure = parser.get_structure('original_structure', f"{pdb_id}.pdb")
original_chain_ids = sorted({chain.id for chain in original_structure.get_chains() if len(chain.id.strip()) == 1 and chain.id.strip() in string.ascii_uppercase})

# 원본의 마지막 체인 ID 찾기
if original_chain_ids:
    last_chain_id = original_chain_ids[-1]
else:
    last_chain_id = 'A'  # 원본 체인이 없을 경우 기본값

# 다음 체인 ID 목록 생성
all_chain_ids = list(string.ascii_uppercase)
start_index = all_chain_ids.index(last_chain_id) + 1
new_chain_ids = all_chain_ids[start_index:]

# 시프트된 구조 로드
shifted_structure = parser.get_structure('shifted_structure', output_translated_pdb)
shifted_chains = [chain for chain in shifted_structure.get_chains() if len(chain.id.strip()) == 1 and chain.id.strip() in string.ascii_uppercase]

# 체인 이름 변경
for chain, new_id in zip(shifted_chains, new_chain_ids):
    chain.id = new_id

# 변경된 구조를 새로운 PDB 파일로 저장
output_renamed_pdb = f"{pdb_id}_renamed_chains.pdb"
io.set_structure(shifted_structure)
io.save(output_renamed_pdb)

print(f"Renamed chains PDB saved to: {output_renamed_pdb}")


Renamed chains PDB saved to: 6cu7_renamed_chains.pdb


In [8]:
from Bio.PDB import PDBParser, PDBIO, Structure, Model

# 5단계: 두 PDB 파일 병합

# 원본 구조 로드
original_pdb_file = f"{pdb_id}.pdb"
renamed_pdb_file = output_renamed_pdb
output_stacked_pdb_file = f"{pdb_id}_stacked.pdb"

original_structure = parser.get_structure('original_structure', original_pdb_file)
renamed_structure = parser.get_structure('renamed_structure', renamed_pdb_file)

# 새로운 구조 생성
merged_structure = Structure.Structure('merged_structure')
merged_model = Model.Model(0)  # 단일 모델 사용
merged_structure.add(merged_model)

# 원본 체인을 새 모델에 추가
for chain in original_structure[0]:
    merged_model.add(chain)

# 이름이 변경된 체인을 새 모델에 추가
for chain in renamed_structure[0]:
    merged_model.add(chain)

# 병합된 구조를 새로운 PDB 파일로 저장
io.set_structure(merged_structure)
io.save(output_stacked_pdb_file)

print(f"Stacked PDB saved to: {output_stacked_pdb_file}")


Stacked PDB saved to: 6cu7_stacked.pdb


In [9]:
# PDB 파일 로드 및 구조 시각화
parser = PDBParser(QUIET=True)
structure = parser.get_structure('structure', output_stacked_pdb_file)

# NGLView로 단백질 구조 시각화
view = nv.show_biopython(structure)
view.add_representation('cartoon')
view

NGLWidget()